In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.cluster as clu
from sklearn import neighbors as nei
from sklearn import metrics as met

# User to User

In [5]:
df = pd.read_csv("train.csv")
df.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [7]:
train_size = round(df.shape[0]*0.999)
test_size = df.shape[0] - round(df.shape[0]*0.999) 
print("train size", train_size)
print("test size", test_size)

train_df = df[:train_size]
print(train_df.shape)
test = df[train_size:]
print(test.shape)

train size 989219
test size 990
(989219, 3)
(990, 3)


In [9]:
U = train_df.pivot("userID","movieID","rating")
U.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
test.head()

,userID,movieID,rating
989219,5847,344,4
989220,5626,3527,3
989221,2798,1544,3
989222,4969,1617,5
989223,4672,3175,5


In [11]:
test = df[train_size:]
test.head()

,userID,movieID,rating
989219,5847,344,4
989220,5626,3527,3
989221,2798,1544,3
989222,4969,1617,5
989223,4672,3175,5


In [14]:
train = U.copy()

In [23]:
myPreds = []
realLabels = []
t = 0
for i in test.index:
    k = 40
    movieId = test.movieID[i] 
    userId  = test.userID[i] 
    print(movieId,userId)
    #if not np.isnan(test.loc[userId, movieId]):
    #if t == 5:
    #    break
    filtered_train = train.dropna(subset=[movieId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    if k >= len(filtered_train):
        k = len(filtered_train)

    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)

    test_row = U.loc[userId:userId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]

    userIds = filtered_train.iloc[indices, :].index
    pred = train.loc[userIds, movieId].values.mean()
    if pred > 5:
        pred = 5
    if pred < 1:
        pred = 1  
    myPreds.append(pred)

    #real_label = test.loc[userId, movieId]
    real_label = test.rating[i]
    print(pred,real_label)
    realLabels.append(real_label)
    t += 1
        

344 5847
3.4 4
3527 5626
3.225 3
1544 2798
2.75 3
1617 4969
4.3 5
3175 4672
3.625 5
1962 4147
4.125 4
1212 3534
4.675 4
3083 965
3.875 3
421 3107
3.45 3
2572 103
3.175 2
2337 5202
3.15 4
653 41
2.975 3
296 2522
4.4 2
20 3184
2.625 3
50 2529
4.75 4
3462 2495
4.4 4
2411 3665
2.075 2
2581 3361
3.075 2
919 5278
4.05 4
2795 5252
4.05 5
2253 4987
2.5 4
3698 2641
3.2 5
2460 5492
2.45 1
1722 708
3.675 4
1527 1906
3.775 5
2443 4144
3.575 2
2058 4152
3.375 2
2483 821
2.9583333333333335 2
1370 276
2.975 3
539 654
3.125 2
3794 3797
3.275 2
3683 1826
4.1 3
1625 2986
3.9 4
3703 2685
3.2 4
1405 1309
3.075 3
1573 5222
3.3 3
1042 3500
3.45 3
1965 4287
3.9 5
1304 1927
3.95 5
707 2233
3.075 2
2002 4009
4.0 5
3173 3920
3.025 5
2432 4121
3.1 2
3033 2861
3.05 3
3114 2540
4.25 5
260 5979
4.4 4
350 5020
3.35 3
1080 566
4.2 5
372 5599
2.925 3
2160 4334
3.65 5
2427 3770
3.4 3
1387 2676
4.35 5
587 5959
3.775 5
1278 428
4.025 4
1291 690
4.0 4
3111 821
3.675 3
923 456
4.675 4
2890 5039
4.3 4
3623 4948
3.625 1
587 

In [24]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.9569431872677802

In [21]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.9591165432116349

In [25]:
U = df.pivot("userID","movieID","rating")
U.head()

movieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,3.0,NaN,4.0,NaN,4.0,NaN,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,3.0,4.0,NaN,NaN,NaN,3.0,NaN,NaN,2.0,4.0,3.0,NaN,4.0,3.0,4.0,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
test = pd.read_csv("test_without_labels.csv",sep="_",names=["userID","movieID"])[1:]
train = U.copy()
test.head(5)


,userID,movieID
1,629,2683.0
2,601,904.0
3,5673,3717.0
4,5616,1721.0
5,1099,3697.0


In [28]:
myPreds = []
realLabels = []
movieIds = []
userIds = []
user_and_movieID = []
t = 0
for i in test.index:
    k = 50
    movieId = int(test.movieID[i] )
    userId  = int(test.userID[i] )
    user_and_movieID.append(str(userId) + "_" + str(movieId))
    #print(movieId,userId)
    #if not np.isnan(test.loc[userId, movieId]):
    #if t == 5:
    #    break
    filtered_train = train.dropna(subset=[movieId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    if k >= len(filtered_train):
        k = len(filtered_train)
        
    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)

    test_row = U.loc[userId:userId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]

    userIds = filtered_train.iloc[indices, :].index
    pred = train.loc[userIds, movieId].values.mean()
    if pred > 5:
        pred == 5
    if pred < 1:
        pred = 1  
    myPreds.append(pred)

    #real_label = test.loc[userId, movieId]
    #real_label = test.rating[i]
    #print(pred,real_label)
    #realLabels.append(real_label)
    t += 1
    if t % 250 == 0:
        print(t)
        

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
3000
3250
3500
3750
4000
4250
4500
4750
5000
5250
5500
5750
6000
6250
6500
6750
7000
7250
7500
7750
8000
8250
8500
8750
9000
9250
9500
9750


In [29]:
for i in range(5):
    print(user_and_movieID[i],myPreds[i])

629_2683 3.02
601_904 4.34
5673_3717 2.7
5616_1721 3.36
1099_3697 2.84


In [30]:
pd.DataFrame( { "IDs": user_and_movieID,
                "rating" : myPreds}).to_csv("submission1.csv",index=False) # RMSE 1.2 because of the error

In [31]:
pd.DataFrame( { "IDs": user_and_movieID,
                "rating" : myPreds}).to_csv("submission2.csv",index=False) # I fixed the error RMSE = 0,9